In [1]:
import numpy as np
import pycyc
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import MultipleLocator
import matplotlib as mpl
from plotting import plot_intrinsic_vs_observed
import copy
import pickle
import importlib
import sys

In [2]:
mpl.rcParams["image.aspect"] = "auto"

In [3]:
from scipy.fft import rfft, fft, fftshift, ifft, fftn, ifftn

CS = pycyc.CyclicSolver("P2067/chan07/53873.27864.07.15s.cyc", zap_edges = 0.05556, pscrunch=True)
CS.data.shape, CS.nspec

CS.load("P2067/chan07/53873.31676.07.15s.cyc")
CS.data.shape, CS.nspec

CS.initProfile(maxsubint=128)

with open ("cs_post_init.pkl", "wb") as fh:
    pickle.dump(CS, fh)

In [ ]:
with open ("cs_post_init.pkl", "rb") as fh:
    CS = pickle.load(fh)

In [ ]:
plt.plot(CS.pp_int)

In [ ]:
# reload a module to incorporate code chanegs

if "pycyc" in sys.modules.keys():
    print("Updated pycyc")
    importlib.reload(sys.modules["pycyc"])
import pycyc

if "fista" in sys.modules.keys():
    print("Updated fista")
    importlib.reload(sys.modules["fista"])
import fista

In [ ]:
# FISTA DA LOT!
CS.iprint = 1
CS.make_plots = 0
CS.niter = 0
CS.initWavefield()

In [ ]:
CS.merit

In [ ]:
CS.h_doppler_delay_grad.shape
plt.plot (np.real(CS.h_time_delay[100]), ".")

In [ ]:
plt.plot (np.real(CS.h_time_delay_grad[0]), ".")

In [ ]:
CS.h_doppler_delay.shape

In [ ]:
y_n = np.copy(CS.h_doppler_delay)
x_n = np.copy(CS.h_doppler_delay)
t_n = 1
demerits = np.array([])
x_n, y_n, L, t_n, demerits = fista.take_fista_step(i=0, func=CS, 
    backtrack=True, alpha=1.0, eta=10, y_n=y_n, _lambda=None,
    delay_for_inf=-int(CS.nchan/2), 
    zero_penalty_coords = np.array([]),
    fix_phase_value = None,
    fix_phase_coords = None,
    fix_support= np.array([]),
    t_n=t_n,
    x_n=x_n,
    demerits = demerits,
    eps = None,
)

In [ ]:
plt.plot (np.real(y_n[0]))
#plt.ylim((-5e-1,5e-1))

In [ ]:
CS.nopt += 1
CS.updateProfile(y_n)
plt.plot(CS.pp_int)

In [ ]:
plt.plot (np.real(y_n[1]))

In [ ]:
CS.updateWavefield ()
CS.merit

x_n, y_n, L, t_n, demerits = fista.take_fista_step(i=1, func=CS, 
    backtrack=True, alpha=1.0/L, eta=1.5, y_n=y_n, _lambda=None,
    delay_for_inf=-int(CS.nchan/2), 
    zero_penalty_coords = np.array([]),
    fix_phase_value = None,
    fix_phase_coords = None,
    fix_support= np.array([]),
    t_n=t_n,
    x_n=x_n,
    demerits = demerits,
    eps = None,
)

Manual experimentation with Lipschitz

In [ ]:
x_n, y_n, L, t_n, demerits = fista.take_fista_step(i=1, func=CS, 
    backtrack=False, alpha=1.5, eta=1.5, y_n=y_n, _lambda=None,
    delay_for_inf=-int(CS.nchan/2), 
    zero_penalty_coords = np.array([]),
    fix_phase_value = None,
    fix_phase_coords = None,
    fix_support= np.array([]),
    t_n=t_n,
    x_n=x_n,
    demerits = demerits,
    eps = None,
)

In [ ]:
plt.imshow(np.real(x_n))
plt.colorbar()

In [ ]:
plt.plot(np.abs(x_n[0]))

In [ ]:
for i in range (100):
    CS.nopt += 1
    
    CS.updateProfile (y_n)
    CS.updateWavefield ()
    
    print(f"**************************************************************** CS.merit={CS.merit}")
    x_n, y_n, L, t_n, demerits = fista.take_fista_step(i=i+2, func=CS, 
        backtrack=False, alpha=1.5, 
        eta=5, y_n=y_n, _lambda=None,
        delay_for_inf=-int(CS.nchan/2), 
        zero_penalty_coords = np.array([]),
        fix_phase_value = None,
        fix_phase_coords = None,
        fix_support= np.array([]),
        t_n=t_n,
        x_n=x_n,
        demerits = demerits,
        eps = None,
    )